In [ ]:
## Project Cardcaptor 
## Project MiniOtter

In [2]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [27]:
origin_df = pd.read_csv("ssg_card.csv", encoding = 'utf-8-sig')
len(origin_df)

12

In [17]:
url = 'https://www.ssg.com/event/eventMain.ssg' 
r = requests.get(url)

if r.status_code == 200:
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')

else : 
    print(r.status_code)

# import requests
# from bs4 import BeautifulSoup

# headers = {'User-Agent': 'Mozilla/5.0'}

# url = "https://linkedin.com/company/1005"

# r = requests.get(url, headers=headers)
# print(r.text)

# soup = BeautifulSoup(r.text, 'html.parser')
# print(soup.prettify())

In [18]:
events = soup.find_all(class_ = 'evt_osmu_unit' )
event_ls = [] 
for event in events : 
    try : 
        name = event.find(class_ = 'eo_tit').find("strong").text
        if "카드" in name : 
            event_ls.append(event)
    except : 
        continue
        #하단 문화이벤트 영역이 공란일 경우 에러 발생시키기 때문에 try except 처리한다 

# [event for event in events if "카드" in event.find(class_ = 'eo_tit').find("strong").text]

In [19]:
df = pd.DataFrame( {"date_start" : []
                    ,"date_end" : [] 
                    ,"card" : []
                    ,"descript" : []
                    ,"platform" : []
                    ,"detail_date" : []
                    ,"detail_card" : []
                    ,"detail_desc" : []
                    ,"detail_goods" : []
                    ,"detail_limit" : []
                    })

In [38]:

main_url = 'https://www.ssg.com'

for event in event_ls : 
    
    #기본정보
    date_total = event.find(class_ = 'eo_period').text.split(" - ")
    card = event.find(class_ = 'eo_tit').find("strong").text
    descript = event.find(class_ = 'desc1').text + " " + event.find(class_ = 'desc2').text
    platform = event.find(class_ = 'eo_mall').text.replace("\n", "")
    
    #링크타고 들어가서 얻는 디테일 
    #아니 왜 링크 유형이 제각각이야 
    event_url = event.find(href=True)["href"]
    if "https://" not in event_url : 
        second_url = main_url + event_url
    else : 
        second_url = event.find(href=True)["href"]
        
    r2 = requests.get(second_url)

    if r2.status_code == 200:
        html = r2.text
        soup = BeautifulSoup(html, 'html.parser')

    else : 
        print(r2.status_code)
        
    try : 
        details = soup.find(class_="ecard_info").find_all("li")
        for d in details : 

            item = d.find(class_="tit").text
            content = d.text.replace(d.find(class_="tit").text, "")

            if item == '행사기간' : 
                detail_date = content
            elif item == '대상카드' : 
                detail_card = content
            elif item == '행사내용' :
                detail_desc = content
            elif item == '대상상품' : 
                detail_goods = content
            else : 
                detail_limit = content
    except : 
        detail_date, detail_card, detail_desc, detail_goods, detail_limit = "", "", "", "", ""

    row = pd.DataFrame.from_dict( [{"date_start" : date_total[0]
                        ,"date_end" : date_total[1]
                        ,"card" : card
                        ,"descript" : descript
                        ,"platform" : platform
                        ,"detail_date" : detail_date
                        ,"detail_card" : detail_card
                        ,"detail_desc" : detail_desc
                        ,"detail_goods" : detail_goods
                        ,"detail_limit" : detail_limit
                        }])
    df = pd.concat([df, row], ignore_index = True)

In [39]:
df.tail()

,date_start,date_end,card,descript,platform,detail_date,detail_card,detail_desc,detail_goods,detail_limit
27,2022.09.10,2022.09.15,SSG.COM카드 Edition2,최대 3만원! 5% 즉시할인,ssg.com,2022. 9. 10 (토) ~ 9. 15 (목) (※승인일시 기준),"SSG.COM카드 Edition2 (※선불,체크,기프트,법인카드 제외)",SSG.COM카드 Edition2로 결제 시 대상상품 금액의 5% 즉시할인,"SSG.COM상품 (※서비스 상품, 상품권 및 일부상품 제외)",결제페이지에서 SSG.COM카드 Edition2를 선택하시면 즉시할인이 적용 된 카...
28,2022.09.08,2022.09.11,롯데카드,SSGPAY로 7만원 이상 결제 시 5% 청구할인,ssg.com,2022. 9. 8 (목) ~ 9. 11 (일) (※승인일시 기준),"롯데카드 (※단, 법인/기프트/체크카드 제외)",SSGPAY로 롯데카드 7만원 이상 결제 시 5% 청구할인,SSG.COM상품 (※서비스 상품/상품권 및 일부상품 제외),1인 1일 최대 5만원 한도
29,2022.09.07,2022.09.12,SSG.COM삼성카드,최대 3만원! 7% 즉시할인,ssg.com,2022. 9. 7 (수) ~ 9. 12 (월) (※승인일시 기준),SSG.COM삼성카드 ※본 프로모션은 SSG.COM삼성카드를 보유하신 기존 회원님께...,SSG.COM삼성카드로 결제 시 7% 즉시할인,"SSG.COM 상품 (※서비스 상품, 상품권 및 일부상품 제외)",결제페이지에서 SSG.COM삼성카드를 선택하시면 즉시할인이 적용 된 카드할인 최적가...
30,2022.07.08,2023.06.28,스마일클럽 전용 카드,스마일클럽 가입비 100% 지원 SSG머니 최대 12% 적립,ssg.com,,,,,
31,2022.03.28,2023.02.16,현대카드,장기무이자 최대 20개월 혜택!,ssg.com,,,,,


In [28]:
df = pd.concat([df, origin_df], ignore_index = True)
df = df.drop_duplicates(subset = ["date_start", "date_end", "card", "descript", "platform", "detail_date", "detail_card", "detail_desc", "detail_goods", "detail_limit"], keep = "first")

In [24]:
df.to_csv("ssg_card.csv", index = False, encoding = 'utf-8-sig')